In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
#!pip install genki_widgets

In [13]:
import os

os.chdir('../')

In [14]:
from genki_signals.signal_sources import MouseSignalSource, CameraSignalSource, Sampler

source = Sampler(
    {
        'mouse_position': MouseSignalSource(),
        'image': CameraSignalSource()
    },
    sample_rate=100
)

In [15]:
from genki_signals.signal_system import SignalSystem
from genki_signals.signal_frontends.visualization import WidgetFrontend, Line, Video

system = SignalSystem(source, [], update_rate=60)

system.start()

In [16]:
video = Video("image")


frontend = WidgetFrontend(system, widgets=[video])

In [17]:
video.widget

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…